In [1]:
%pwd

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier\\research'

In [2]:
# src\CNNClassifier\entity\config_entity.py
from dataclasses import dataclass
from pathlib import Path

# Creating decorator and it is creating a immutable dataclass
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    Source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [3]:
# setting location so that we can import CNNClassifier module
import sys
import os

# Navigate two levels up to the project root directory (adjust if necessary)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# Add the project root to sys.path
sys.path.append(project_root)

# Now, the CNNClassifier module should be found
project_root

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier\\src'

In [10]:
# Configuration component

# src\CNNClassifier\config\configuration.py
from CNNClassifier.utils.utils import read_yaml , create_dir
from CNNClassifier.constants import CONFIG_FILE_PATH , PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(self , config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)    # reading the files which are define in 'constants'
        self.params = read_yaml(params_filepath)
        create_dir([Path(self.config.artifacts_root)])  #  Creating artifact directory

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dir([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            Source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [15]:
# Data Ingestion Component

#src\CNNClassifier\components\stage_01_data_ingestion.py
import os
import urllib.request as request    # For sending request to the server
from zipfile import ZipFile
from CNNClassifier import logger    # When we import from CNNClassifier it will 1st run '__init__.py'
from tqdm import tqdm
from CNNClassifier.entity.config_entity import DataIngestionConfig
# For data ingestion

class DataIngestion:
    def __init__(self , config:DataIngestionConfig):  # type of config -> DataIngestionConfig
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file): # if it doesn't exists
            logger.info('trying to download file')
            request.urlretrieve(
                url = self.config.Source_URL,
                filename = self.config.local_data_file
            )    # request for data
        else:
            logger.info('file already exists')

    def get_updated_list_of_files(self , list_of_files):
        return [f for f in list_of_files if f.endswith('.jpg')]

    def preprocess(self , zf , f , working_dir):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f , working_dir)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file , mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self.get_updated_list_of_files(list_of_files)
            for f in tqdm(updated_list_of_files):    # tqdm means we are going to showcase the progress
                self.preprocess(zf , f , self.config.unzip_dir)

In [12]:
%pwd

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier'

In [7]:
#changing current directory so that we can import modules easily
import os
os.chdir('../')
%pwd

'c:\\Users\\Vishwamohan\\OneDrive\\Desktop\\Vishu\\Projects\\CNN_Classifier'

In [13]:
print(os.path.exists('configs/config.yaml'))   # the directory path exists

True


In [16]:
# Running all the steps using pipeline

#src\CNNClassifier\components\stage_01_data_ingestion.py
logger.info(f'data ingestion started')

config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()   # For getting configuration 

data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()

data_ingestion.unzip_and_clean()

logger.info(f'data ingestion completed')

[2024-10-08 22:55:28,730: INFO: 3464244792]: data ingestion started
[2024-10-08 22:55:28,735: INFO: utils]: create directory at : artifacts
[2024-10-08 22:55:28,740: INFO: utils]: create directory at : artifacts\data_ingestion
[2024-10-08 22:55:28,742: INFO: 4059024009]: file already exists


100%|██████████| 25000/25000 [01:39<00:00, 250.95it/s]

[2024-10-08 22:57:08,644: INFO: 3464244792]: data ingestion completed
